In [18]:
import pandas as pd
import numpy as np
import nltk
from collections import defaultdict
from collections import Counter
from src.classes.database.offer import MongoOffer

tokenizer = nltk.RegexpTokenizer(r'\w+')

In [4]:
token_list = [tokenizer.tokenize(x.selling) for x in MongoOffer.objects()]
tagged = nltk.pos_tag(token_list[0])

In [44]:
def get_all_words(token_list: list[list]) -> list:
    words = set()
    for sentence in token_list:
        for word in sentence:
            words.add(word)
    return sorted(words)

def next_word_occurrence(key: str, look_ahead: int, tokens: list[list]) -> dict:
    counter = defaultdict(int)
    for sentence in tokens:
        for x in range(0, len(sentence)):
            if key == sentence[x]: #if we find the key in a sentence
                if (x+look_ahead) < len(sentence):
                    counter[sentence[x+look_ahead]] += 1
    return counter

def calculate_all_probabilities(tokens: list[list]) -> dict:
    all_words = get_all_words(tokens)

    #Generating a list of all words
    word_canvas = {}
    for word in all_words:
        word_canvas[word] = None

    p_word = word_canvas.copy()
    for word in p_word:
        p_word[word] = {'word': word, 'p': word_canvas.copy()}

    return p_word

In [45]:
calculate_all_probabilities(token_list)['0']

{'word': '0',
 'p': {'0': None,
  '001': None,
  '1': None,
  '10': None,
  '100': None,
  '1000': None,
  '1000xm4': None,
  '10100f': None,
  '10105': None,
  '10400': None,
  '1050': None,
  '1070': None,
  '10700k': None,
  '10700kf': None,
  '1070ti': None,
  '1080': None,
  '1080ti': None,
  '10870h': None,
  '10900k': None,
  '10gb': None,
  '10uh': None,
  '11': None,
  '1135g7': None,
  '11400': None,
  '11400f': None,
  '1165g7': None,
  '11700': None,
  '11800h': None,
  '11th': None,
  '12': None,
  '120hz': None,
  '120mm': None,
  '12600k': None,
  '12700f': None,
  '12700k': None,
  '128gb': None,
  '12900k': None,
  '12tb': None,
  '12th': None,
  '13': None,
  '1300x': None,
  '13600k': None,
  '13700k': None,
  '14': None,
  '140w': None,
  '1440p': None,
  '144hz': None,
  '14in': None,
  '15': None,
  '16': None,
  '1600': None,
  '1650': None,
  '165hz': None,
  '1660': None,
  '168gb': None,
  '16gb': None,
  '16gbx2': None,
  '16gvr': None,
  '17': None,
  '1700'

In [22]:
test = Counter(get_all_words(token_list))
test.update(0)

TypeError: 'int' object is not iterable

In [6]:
next_word_occurrence('i7', 1, token_list)

defaultdict(int,
            {'12700k': 4,
             '10870h': 1,
             '11800h': 1,
             '16gb': 1,
             '12th': 2,
             '8700k': 3,
             '9700': 1,
             '10700k': 1,
             '13700k': 1,
             '7700k': 1})

In [7]:
class Sentence:
    def __init__(self):


class Word:
    def __init__(self, word: str):
        self.word = word
        self.p_of_next_word = [next_word_occurrence(word, x, token_list) for x in range(1,6)]


In [8]:
test = Word('i5')

In [9]:
test.p_of_next_word[1]

defaultdict(int,
            {'16gb': 2,
             'evga': 1,
             'z87m': 1,
             'new': 1,
             'gen': 1,
             'docking': 1,
             '1x': 2,
             '1070ti': 1,
             'mobo': 1,
             'office': 1,
             '15': 1,
             'i5': 1,
             'asus': 1})

p_of_next_word['word']['looking_for_word'][INDEX HERE OF LOOKAHEAD]
return object like {occurrences: 1, probability: 0.5}

word1
    looking_for1
        0: {occurenecs:1, probability: 0.5}
        1:  {occurenecs:1, probability: 0.5}
        2:  {occurenecs:0, probability: 5}
    looking_for2